In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import StratifiedKFold

# Load pheno

In [2]:
pheno_p = '/Users/harveyaa/Documents/masters/data/pheno_26-01-22.csv'
pheno = pd.read_csv(pheno_p, index_col=0)

/var/folders/5x/9dwn0lbx633gccy22ts5bskh0000gn/T/ipykernel_94905/1011432404.py:2: DtypeWarning: Columns (7,8,12,13,14,19,20,24,27,31,42,43,44,49,51,59,60,62,64,65,68,101,121,163) have mixed types. Specify dtype option on import or set low_memory=False.
  pheno = pd.read_csv(pheno_p, index_col=0)


# Load datasets

In [8]:
cases = ['ADHD','ASD','BIP','SZ','DEL1q21_1','DUP1q21_1','DEL22q11_2','DUP22q11_2','DEL16p11_2','DUP16p11_2']
id_p = '/Users/harveyaa/Documents/masters/neuropsych_mtl/datasets/{}.txt'
ids = []
for case in cases:
    ids.append(pd.read_csv(id_p.format(case),header=None))
ids = dict(zip(cases,ids))

# Get CV splits

In [20]:
cvs = []
for case in cases:
    print(case)
    df = pheno[pheno.index.isin(ids[case][0])]

    skf = StratifiedKFold(n_splits=5,shuffle=True)
    res = skf.split(df.index,df['SITE'])

    cols = []
    for _,r in res:
        col = np.zeros(len(df.index))
        col[r] = 1
        cols.append(col)
    splits = pd.DataFrame(cols,columns=df.index, index=['fold_0','fold_1','fold_2','fold_3','fold_4']).transpose()
    splits.to_csv(f'{case}.csv')
    cvs.append(splits)

ADHD
ASD
BIP
SZ
DEL1q21_1
DUP1q21_1
DEL22q11_2
DUP22q11_2
DEL16p11_2
DUP16p11_2


/Users/harveyaa/miniconda3/envs/fmri/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/Users/harveyaa/miniconda3/envs/fmri/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/harveyaa/miniconda3/envs/fmri/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/harveyaa/miniconda3/envs/fmri/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/harveyaa/miniconda3/envs/fmri/lib/python3.9/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated

# Investigate splits

In [32]:
ratios = []
for i,case in enumerate(cases):
    cv = cvs[i]

    ratio = []
    for fold in [0,1,2,3,4]:
        subs = cv[cv[f'fold_{fold}']==1].index
        ratio.append(pheno[pheno.index.isin(subs)][case].mean())
    ratios.append(ratio)

In [37]:
df = pd.DataFrame(ratios,index=cases)
df['mean'] = df.mean(axis=1)

df

,0,1,2,3,4,mean
ADHD,0.353448,0.443478,0.330435,0.452174,0.356522,0.387211
ASD,0.470899,0.539683,0.507937,0.446809,0.537234,0.500512
BIP,0.312500,0.250000,0.193548,0.322581,0.322581,0.280242
SZ,0.500000,0.398438,0.500000,0.425197,0.393701,0.443467
DEL1q21_1,0.600000,0.500000,0.600000,0.500000,0.300000,0.500000
DUP1q21_1,0.375000,0.625000,0.750000,0.428571,0.285714,0.492857
DEL22q11_2,0.611111,0.588235,0.529412,0.294118,0.470588,0.498693
DUP22q11_2,0.777778,0.444444,0.555556,0.333333,0.375000,0.497222
DEL16p11_2,0.461538,0.769231,0.461538,0.461538,0.333333,0.497436
DUP16p11_2,0.428571,0.714286,0.214286,0.571429,0.571429,0.500000
